In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import math

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df_0 = pd.read_csv('Data\Concrete_Data_Yeh.csv')
df_0.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cement            1030 non-null   float64
 1   slag              1030 non-null   float64
 2   flyash            1030 non-null   float64
 3   water             1030 non-null   float64
 4   superplasticizer  1030 non-null   float64
 5   coarseaggregate   1030 non-null   float64
 6   fineaggregate     1030 non-null   float64
 7   age               1030 non-null   int64  
 8   csMPa             1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [5]:
df_p = preprocessing.normalize(df_0, axis=0)
df = pd.DataFrame(df_p, columns=df_0.columns)
df.describe()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,0.029208,0.020274,0.020141,0.030946,0.022451,0.031060,0.030993,0.018259,0.028241
std,0.010856,0.023672,0.023786,0.003640,0.021616,0.002482,0.003212,0.025260,0.013172
min,0.010596,0.000000,0.000000,0.020759,0.000000,0.025571,0.023798,0.000400,0.001837
25%,0.019984,0.000000,0.000000,0.028105,0.000000,0.029754,0.029285,0.002799,0.018694
50%,0.028349,0.006036,0.000000,0.031531,0.023158,0.030903,0.031230,0.011197,0.027158
75%,0.036359,0.039220,0.043969,0.032724,0.036908,0.032863,0.033013,0.022393,0.036375
max,0.056096,0.098607,0.074372,0.042098,0.116515,0.036554,0.039768,0.145956,0.065126


<h1>Selección de variables</h1>

In [6]:
## Generación de realciones

columnas = list(df.drop(columns=['csMPa']).columns)
for i in columnas:
    columnas_2 = [x for x in columnas if x not in i]
    for j in columnas_2:
        nombre_columna = i + '-' + j
        df[nombre_columna] = df[i] / df[j]

## Dejar varibales con valores no indeterminados

df = df[list(df.describe().replace([np.inf, -np.inf], np.nan).iloc[-1].dropna().index)]

D:\Anaconda\lib\site-packages\numpy\lib\function_base.py:3949: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


In [7]:
## Generación de Exponenciales

columnas = list(df.drop(columns=['csMPa']).columns)

for i in columnas:
    nombre_columna = 'exp_' + i
    df[nombre_columna] = df[i].apply(lambda x: math.exp(x))

In [10]:
## Generar Dataframe aleatorio para Seleccion de Variables

np.random.seed(1984)

explicativas = list(df.drop(columns=['csMPa']).columns)
df_explicativas = pd.DataFrame(np.random.randint(2,size=(500, len(explicativas))), columns=explicativas)
df_explicativas['aic'] = 200000

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,cement-water,cement-coarseaggregate,cement-fineaggregate,cement-age,slag-cement,slag-water,slag-coarseaggregate,slag-fineaggregate,slag-age,flyash-cement,flyash-water,flyash-coarseaggregate,flyash-fineaggregate,flyash-age,water-cement,water-coarseaggregate,water-fineaggregate,water-age,superplasticizer-cement,superplasticizer-water,superplasticizer-coarseaggregate,superplasticizer-fineaggregate,superplasticizer-age,coarseaggregate-cement,coarseaggregate-water,coarseaggregate-fineaggregate,coarseaggregate-age,fineaggregate-cement,fineaggregate-water,fineaggregate-coarseaggregate,fineaggregate-age,age-cement,age-water,age-coarseaggregate,age-fineaggregate,exp_cement,exp_slag,exp_flyash,exp_water,exp_superplasticizer,exp_coarseaggregate,exp_fineaggregate,exp_age,exp_cement-water,exp_cement-coarseaggregate,exp_cement-fineaggregate,exp_cement-age,exp_slag-cement,exp_slag-water,exp_slag-coarseaggregate,exp_slag-fineaggregate,exp_slag-age,exp_flyash-cement,exp_flyash-water,exp_flyash-coarseaggregate,exp_flyash-fineaggregate,exp_flyash-age,exp_water-cement,exp_water-coarseaggregate,exp_water-fineaggregate,exp_water-age,exp_superplasticizer-cement,exp_superplasticizer-water,exp_superplasticizer-coarseaggregate,exp_superplasticizer-fineaggregate,exp_superplasticizer-age,exp_coarseaggregate-cement,exp_coarseaggregate-water,exp_coarseaggregate-fineaggregate,exp_coarseaggregate-age,exp_fineaggregate-cement,exp_fineaggregate-water,exp_fineaggregate-coarseaggregate,exp_fineaggregate-age,exp_age-cement,exp_age-water,exp_age-coarseaggregate,exp_age-fineaggregate,aic
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.0000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.0000,500.000000,500.000000,500.0000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.0000,500.000000,500.000000,500.000000,500.0000,500.000000,500.0
mean,0.532000,0.508000,0.494000,0.496000,0.524000,0.472000,0.502000,0.526000,0.474000,0.474000,0.496000,0.516000,0.502000,0.506000,0.500000,0.524000,0.472000,0.530000,0.486000,0.458000,0.496000,0.532000,0.474000,0.496000,0.522000,0.496000,0.482000,0.4800,0.496000,0.472000,0.500000,0.484000,0.504000,0.474000,0.552000,0.504000,0.508000,0.476000,0.484000,0.510000,0.500000,0.510000,0.484000,0.508000,0.506000,0.508000,0.506000,0.476000,0.474000,0.460000,0.492000,0.512000,0.510000,0.528000,0.534000,0.548000,0.500000,0.512000,0.4800,0.514000,0.498000,0.4800,0.492000,0.502000,0.508000,0.470000,0.448000,0.524000,0.492000,0.500000,0.504000,0.514000,0.528000,0.486000,0.532000,0.542000,0.486000,0.482000,0.454000,0.478000,0.4800,0.516000,0.508000,0.492000,0.5200,0.510000,200000.0
std,0.499475,0.500437,0.500465,0.500485,0.499924,0.499715,0.500497,0.499824,0.499824,0.499824,0.500485,0.500244,0.500497,0.500465,0.500501,0.499924,0.499715,0.499599,0.500305,0.498732,0.500485,0.499475,0.499824,0.500485,0.500016,0.500485,0.500176,0.5001,0.500485,0.499715,0.500501,0.500244,0.500485,0.499824,0.497787,0.500485,0.500437,0.499924,0.500244,0.500401,0.500501,0.500401,0.500244,0.500437,0.500465,0.500437,0.500465,0.499924,0.499824,0.498897,0.500437,0.500357,0.500401,0.499715,0.499342,0.498189,0.500501,0.500357,0.5001,0.500305,0.500497,0.5001,0.500437,0.500497,0.500437,0.499599,0.497787,0.499924,0.500437,0.

In [12]:
## Definición funcion a optimizar

def aic_criterion(df):
    n_var = len(df.drop(columns=['residuals'
                                 , 'csMPa'
                                 , 'csMPa_est']).columns)
    
    n = len(df)
    
    res = sum(df['residuals'].apply(lambda x: x ** 2))
    
    return((n * (math.log(2 * math.pi) + 1 + math.log(res / n))) + ((n_var + 1) * 2))

In [13]:
## Generacion de Población Inicial

y = df['csMPa']
aic_col = pd.Series([])

for i in range(len(df_explicativas)):
    X = df[list(df_explicativas.iloc[i][df_explicativas.iloc[i] == 1].index)]
    X_val, X_other, y_val, y_other = train_test_split(X, y, test_size=0.80, random_state=42, shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(X_other, y_other, test_size=0.40, shuffle=False)

    del X_other
    del y_other
    
    reg = LinearRegression().fit(X_train, y_train)
    y_val_est = pd.Series(reg.predict(X_val))
    
    df_val = pd.concat([X_val, y_val,y_val_est], axis=1).rename(columns={0 : 'csMPa_est'})
    df_val['residuals'] = df_val['csMPa'] - df_val['csMPa_est']
    
    df_explicativas['aic'].iloc[i] = aic_criterion(df_val)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWit

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

...

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,cement-water,cement-coarseaggregate,cement-fineaggregate,cement-age,slag-cement,slag-water,slag-coarseaggregate,slag-fineaggregate,slag-age,flyash-cement,flyash-water,flyash-coarseaggregate,flyash-fineaggregate,flyash-age,water-cement,water-coarseaggregate,water-fineaggregate,water-age,superplasticizer-cement,superplasticizer-water,superplasticizer-coarseaggregate,superplasticizer-fineaggregate,superplasticizer-age,coarseaggregate-cement,coarseaggregate-water,coarseaggregate-fineaggregate,coarseaggregate-age,fineaggregate-cement,fineaggregate-water,fineaggregate-coarseaggregate,fineaggregate-age,age-cement,age-water,age-coarseaggregate,age-fineaggregate,exp_cement,exp_slag,exp_flyash,exp_water,exp_superplasticizer,exp_coarseaggregate,exp_fineaggregate,exp_age,exp_cement-water,exp_cement-coarseaggregate,exp_cement-fineaggregate,exp_cement-age,exp_slag-cement,exp_slag-water,exp_slag-coarseaggregate,exp_slag-fineaggregate,exp_slag-age,exp_flyash-cement,exp_flyash-water,exp_flyash-coarseaggregate,exp_flyash-fineaggregate,exp_flyash-age,exp_water-cement,exp_water-coarseaggregate,exp_water-fineaggregate,exp_water-age,exp_superplasticizer-cement,exp_superplasticizer-water,exp_superplasticizer-coarseaggregate,exp_superplasticizer-fineaggregate,exp_superplasticizer-age,exp_coarseaggregate-cement,exp_coarseaggregate-water,exp_coarseaggregate-fineaggregate,exp_coarseaggregate-age,exp_fineaggregate-cement,exp_fineaggregate-water,exp_fineaggregate-coarseaggregate,exp_fineaggregate-age,exp_age-cement,exp_age-water,exp_age-coarseaggregate,exp_age-fineaggregate,aic
0,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1,0,0,0,1,1,0,1,1,0,-860.819933
1,0,0,1,1,0,1,0,0,0,0,1,0,1,1,1,1,1,0,0,1,0,1,1,1,1,0,1,0,0,1,0,1,1,0,1,0,1,1,0,0,1,0,1,0,1,1,1,1,0,1,1,1,1,0,1,1,1,0,1,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,1,0,0,0,-857.699568
2,1,0,0,1,1,1,0,0,0,0,0,1,1,1,1,0,0,0,1,0,0,1,1,1,1,0,1,1,0,0,1,1,1,1,0,1,0,1,1,1,1,0,0,0,1,1,0,0,1,0,1,0,1,1,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,1,0,0,0,1,1,1,1,-973.244687
3,0,0,0,1,1,0,1,1,1,0,0,0,1,0,0,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1,1,0,0,0,1,1,0,1,0,1,1,1,0,0,1,0,1,0,1,0,0,1,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,19943.348409
4,0,1,0,1,1,0,0,0,1,0,1,1,0,1,0,0,1,1,1,1,1,0,0,1,1,0,0,1,1,1,1,1,1,1,0,1,1,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,1,1,1,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,1,1,1,0,1,0,0,0,1,0,1,1,1,0,0,19831.336605
